<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_ResNet/test_sample_ResNet101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Wed Jan 13 15:06:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git
%cd PyTorch-Architectures/modeling_ResNet/

Cloning into 'PyTorch-Architectures'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 1324 (delta 10), reused 11 (delta 4), pack-reused 1303
Receiving objects: 100% (1324/1324), 8.54 MiB | 21.12 MiB/s, done.
Resolving deltas: 100% (778/778), done.
/content/PyTorch-Architectures/modeling_ResNet


In [3]:
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

from torchvision import datasets
from torchvision import transforms

from model_large import ResNet, Bottleneck

if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True

In [4]:
# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
EPOCHS = 2

# Architecture
NUM_FEATURES = 128 * 128
NUM_CLASSES = 10
BATCH_SIZE = 128
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
GRAYSCALE = True

In [ ]:
custom_transform = transforms.Compose([transforms.Resize((128, 128)),
                                       transforms.ToTensor()]) # --> Done to accomodate large architecture

train_dataset = datasets.MNIST(
    root='data',
    train=True,
    transform=custom_transform,
    download=True,
)
test_dataset = datasets.MNIST(
    root='data',
    train=False,
    transform=custom_transform,
)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [6]:
# Check Dataloaders dimensions
for features, targets in train_loader:
  print('Image Dimensions: ', features.shape)
  print('Label Dimensions: ', targets.shape)
  break

Image Dimensions:  torch.Size([128, 1, 128, 128])
Label Dimensions:  torch.Size([128])


In [7]:
# Model
torch.manual_seed(RANDOM_SEED)

def resnet50(num_classes, grayscale):
  model = ResNet(
      block=Bottleneck,
      layers=[3, 4, 23, 3],
      num_classes=NUM_CLASSES,
      grayscale=GRAYSCALE,
  )
  return model

model = resnet50(NUM_CLASSES, GRAYSCALE)
model.to(DEVICE)

params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Total Trainable Parameters: ', params)

Total Trainable Parameters:  42575818


In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [9]:
def compute_accuracy(model, data_loader, device):
  correct_pred, num_examples = 0, 0
  for i, (features, targets) in enumerate(data_loader):
    features = features.to(DEVICE)
    targets = targets.to(DEVICE)
    logits, probas = model(features)
    _, predicted_labels = torch.max(probas, 1)
    correct_pred += (predicted_labels == targets).sum()
    num_examples += targets.size(0)
  return correct_pred.float() / num_examples * 100

start_time = time.time()
for epoch in range(EPOCHS):
  model.train()
  for idx, (features, targets) in enumerate(train_loader):
    features = features.to(DEVICE)
    targets = targets.to(DEVICE)

    logits, probas = model(features)

    optimizer.zero_grad()
    loss = F.cross_entropy(logits, targets)
    loss.backward()
    optimizer.step()

    # LOGGING
    if idx % 50 == 0:
      print('Epoch: %04d/%04d || Batch: %04d/%04d || Loss: %.3f' % (epoch+1, EPOCHS, idx, len(train_loader), loss.item()))
  
  model.eval()
  with torch.set_grad_enabled(False):
    train_accuracy = compute_accuracy(model, train_loader, DEVICE)
    test_accuracy = compute_accuracy(model, test_loader, DEVICE)
    print('Train Accuracy: %.2f%%' % (train_accuracy))
    print('Test Accuracy: %.2f%%' % (test_accuracy))
  epoch_elapsed_time = (time.time() - start_time) / 60
  print('Epoch Elapsed Time: %.2f min' % (epoch_elapsed_time))
total_training_time = (time.time() - start_time) / 60
print('Total Training Time: %.2f min' % (total_training_time))

Epoch: 0001/0002 || Batch: 0000/0469 || Loss: 2.340
Epoch: 0001/0002 || Batch: 0050/0469 || Loss: 0.274
Epoch: 0001/0002 || Batch: 0100/0469 || Loss: 0.213
Epoch: 0001/0002 || Batch: 0150/0469 || Loss: 0.123
Epoch: 0001/0002 || Batch: 0200/0469 || Loss: 0.087
Epoch: 0001/0002 || Batch: 0250/0469 || Loss: 0.131
Epoch: 0001/0002 || Batch: 0300/0469 || Loss: 0.108
Epoch: 0001/0002 || Batch: 0350/0469 || Loss: 0.061
Epoch: 0001/0002 || Batch: 0400/0469 || Loss: 0.073
Epoch: 0001/0002 || Batch: 0450/0469 || Loss: 0.061
Train Accuracy: 95.07%
Test Accuracy: 94.92%
Epoch Elapsed Time: 8.10 min
Epoch: 0002/0002 || Batch: 0000/0469 || Loss: 0.029
Epoch: 0002/0002 || Batch: 0050/0469 || Loss: 0.118
Epoch: 0002/0002 || Batch: 0100/0469 || Loss: 0.092
Epoch: 0002/0002 || Batch: 0150/0469 || Loss: 0.037
Epoch: 0002/0002 || Batch: 0200/0469 || Loss: 0.085
Epoch: 0002/0002 || Batch: 0250/0469 || Loss: 0.023
Epoch: 0002/0002 || Batch: 0300/0469 || Loss: 0.025
Epoch: 0002/0002 || Batch: 0350/0469 || Lo